# Terminology project 

## **Spacy trainable pipe**

### AGUIAR Mathilde NIAOURI Dimitra

#### M2 NLP

# Goal and tools 

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#### Imports and packages 

In [ ]:
!pip3 install 'spacy==3.4.3'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!python -m spacy download en_core_web_sm
!pip install spacy-transformers

## Uncomment if you want to download one of the following model
# !python -m spacy download en_core_web_md
# !python -m spacy download en_core_web_lg
# !python -m spacy download en_core_web_trf

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 12.8 MB 473 kB/s 
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 53 kB 2.6 MB/s 
     |████████████████████████████████| 4.7 MB 55.0 MB/s 
     |████████████████████████████████| 1.1 MB 66.0 MB/s 
     |████████████████████████████████| 6.6 MB 64.0 MB/s 
     |████████████████████████████████| 182 kB 60.2 MB/s 


In [ ]:
import spacy
import fnmatch
import os
import re
from os import path, walk
import shutil
import glob
import pandas as pd
from spacy.scorer import Scorer
from spacy.training import Example
from spacy.tokens import Doc

Downloading the curated dataset 

In [ ]:
!wget https://raw.githubusercontent.com/MathildeAguiar/terminology_project/tree/main/data

# Data Preprocessing

Helpers to find files and create folders

In [ ]:
def create_folder(path):
    """Create a folder if it doesn't already exist"""
    if not os.path.isdir(path):
        os.makedirs(path)
    return


def find_files(directory, pattern='*.final'):
    """Recursively finds all files matching the pattern."""
    files = []
    for root, dirnames, filenames in walk(directory):
        for filename in fnmatch.filter(filenames, pattern):
            files.append(path.join(root, filename))
    # sort the list, to avoid mismatch in the output files
    files = sorted(files)
    return files


Adapt the format of the IOB tags to be supported by Spacy. Spacy expects tags in the following format: 
```
B-ATTRIBUTE
```
So we convert the `B` into `B-NLP` and the `I` to `I-NLP`. 

In [ ]:
def mapper(token):
  """
  Adapts the tags into a Spacy readable format
  """
  if token == "B":
    return "B-NLP"
  elif token == "I":
    return "I-NLP"
  else: 
    return token

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob

In [ ]:
def retag_files():  
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the training files
    directory_tg_train = "/content/drive/MyDrive/Terminology_project/curated/training_data/" 
    all_training_files = find_files(directory_tg_train) 
    for f in all_training_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_train+"iob/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue 
                new_file.write(new_txt)
            new_file.close()

    # 2nd process the dev files
    directory_tg_dev = "/content/drive/MyDrive/Terminology_project/curated/dev_data/"  
    all_dev_files = find_files(directory_tg_dev)
    for f in all_dev_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_dev+"iob/"+tmp[0].split(".")[0]+".iob"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue 
                new_file.write(new_txt)
            new_file.close()


In [ ]:
retag_files()

### Convert the IOB files into *.spacy* binary files

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/training_data/iob/output 
!mkdir /content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output 

In [ ]:
## For the training set ##
# Uncomment the line you want in function of the model you are using 
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_sm /content/drive/MyDrive/Terminology_project/curated/training_data/iob /content/drive/MyDrive/Terminology_project/curated/training_data/iob/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/training_data/iob /curated/training_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /curated/training_data/iob /curated/training_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/training_data/iob /curated/training_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob/output/2009-35-1-29-46.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob/output/2009-35-1-3-28.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/training_data/iob/output/2009-35-1-47-59.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/tr

In [ ]:
## For the dev set ##
# Uncomment the line you want in function of the model you are using 
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_sm /content/drive/MyDrive/Terminology_project/curated/dev_data/iob /content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output

#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_md /curated/dev_data/iob /curated/dev_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_lg /curated/dev_data/iob /curated/dev_data/iob/output
#!python -m spacy convert -c iob -s -n 10000 -b en_core_web_trf /curated/dev_data/iob /curated/dev_data/iob/output

ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output/2009-35-4-597-635.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output/2010-36-1-1-30.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output/2011-37-3-421-454.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/dev_data/iob/

# Training the pipeline

First convert the base-config into a config file and then simply using the command line `spacy train`

In [ ]:
# If the GPU is available run this cell 
!python -m spacy init fill-config /content/drive/MyDrive/Terminology_project/base_config_CPU2.cfg config.cfg

✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/output_clean  

mkdir: cannot create directory ‘/content/drive/MyDrive/Terminology_project/output_clean’: File exists


In [ ]:
!python -m spacy train /content/config.cfg --output /content/drive/MyDrive/Terminology_project/output_clean --paths.train /content/drive/MyDrive/Terminology_project/curated/training_data/iob/output --paths.dev /content/drive/MyDrive/Terminology_project/curated/dev_data/iob/output

ℹ Saving to output directory:
/content/drive/MyDrive/Terminology_project/output_clean
ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

=========================== Initializing pipeline ===========================
[2022-12-03 11:14:01,089] [INFO] Set up nlp object from config
INFO:spacy:Set up nlp object from config
[2022-12-03 11:14:01,100] [INFO] Pipeline: ['tok2vec', 'tagger', 'ner']
INFO:spacy:Pipeline: ['tok2vec', 'tagger', 'ner']
[2022-12-03 11:14:01,103] [INFO] Created vocabulary
INFO:spacy:Created vocabulary
[2022-12-03 11:14:01,104] [INFO] Finished initializing nlp object
INFO:spacy:Finished initializing nlp object
[2022-12-03 11:14:02,715] [INFO] Initialized pipeline components: ['tok2vec', 'tagger', 'ner']
INFO:spacy:Initialized pipeline components: ['tok2vec', 'tagger', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'tagger', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS

You should obtain 2 models: `model-best` and `model-base`. To test the trained Pipeline it's better to use `model-best`. 

# Testing on the test dataset

### Preprocessinf the test files 

In [ ]:
!mkdir /content/drive/MyDrive/Terminology_project/curated/test_data/conv

In [ ]:
def retag_test_files():
    """ 
    Retag the original .final files by using the Spacy tag format.
    Save the result into a .iob file.    
    """
    # First process the test files
    directory_tg_test = "/content/drive/MyDrive/Terminology_project/curated/test_data/"  
    all_test_files = glob.glob(directory_tg_test + "/*.final")
    for f in all_test_files:
        with open(f, mode='r', encoding='utf-8') as myfile:
            lines = myfile.readlines()
            tmp = re.findall('\d{4}-\S*', f)
            iob_file_name = directory_tg_test+"conv/"+tmp[0].split(".")[0]+".final"
            new_file = open(iob_file_name, "a")
            for l in lines:
              if not l.isspace():
                words = l.split()
                new_array = list(map(mapper, words))
                if len(new_array)>1:
                    new_txt = new_array[0]+"\t"+new_array[1]+"\n"
                elif len(new_array)>0:
                    new_txt = new_array[0]+"\n"
                else: continue 
                new_file.write(new_txt)
            new_file.close()

In [ ]:
retag_test_files()

In [ ]:
path = r'/content/drive/MyDrive/Terminology_project/curated/test_data/conv'
all_files = glob.glob(path + "/*.final")

l = []
cols = ['Token', 'Tag']

for filename in all_files:
    df = pd.read_csv(filename, names = cols,on_bad_lines='skip', sep='\s+', engine='python')
    l.append(df)

# Our tokens from test set are stored in a Dataframe
test_df = pd.concat(l, axis=0, ignore_index=True)

# drop nan values
test_df = test_df.dropna()
test_df['Token'].fillna('', inplace=True)


## Using Scorer to test on the test sentences 

In [ ]:
test_df.head()
test_df.iloc[88]
print(len(test_df['Token']))
print(len(test_df['Tag']))

2772
2772


### Using the command line

In [ ]:
!python -m spacy convert -c iob -s -n 10000 -b en_core_web_sm /content/drive/MyDrive/Terminology_project/curated/test_data/conv /content/drive/MyDrive/Terminology_project/curated/test_data/conv/output


ℹ Auto-detected token-per-line NER format
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/conv/output/2009-35-4-559-595.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/conv/output/2009-35-4-559-595.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_data/conv/output/2010-36-1-31-69.spacy
ℹ Grouping every 10000 sentences into a document.
ℹ Segmenting sentences with parser from model 'en_core_web_sm'.
✔ Generated output file (1 documents):
/content/drive/MyDrive/Terminology_project/curated/test_d

In [ ]:
!python -m spacy evaluate /content/drive/MyDrive/Terminology_project/output_clean/model-best /content/drive/MyDrive/Terminology_project/curated/test_data/conv/output

ℹ Using CPU
ℹ To switch to GPU 0, use the option: --gpu-id 0

================================== Results ==================================

TOK     -     
TAG     100.00
NER P   57.05 
NER R   53.61 
NER F   55.28 
SPEED   24486 


=============================== NER (per type) ===============================

          P       R       F
NLP   57.05   53.61   55.28

